# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.57,83,40,4.07,PN,1698379170
1,1,coquimbo,-29.9533,-71.3436,13.85,76,100,3.09,CL,1698379130
2,2,grytviken,-54.2811,-36.5092,1.62,97,100,6.42,GS,1698379171
3,3,port moresby,-9.4431,147.1797,30.68,66,75,5.14,PG,1698379171
4,4,waitangi,-43.9535,-176.5597,11.01,77,81,4.92,NZ,1698379171


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
pip install cartopy geoviews pyproj

     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ---------------------------------------- 56.0/56.0 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.2 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 0.3/11.2 MB 3.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/11.2 MB 4.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.2 MB 3.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.2 MB 3.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.2 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.2 MB 3.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.4/11.2 MB 3.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/11.2 MB 4.1 MB/s eta 0:00:03
   ------ --------------------------------- 1.7/11.2 MB 3.9 MB/s eta 0:00:03
   -----

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

# Define a function to create a plot with points representing cities and size representing humidity
def create_city_map(data):
    return data.hvplot.points(
        geo=True,
        x='Lng',
        y='Lat',
        size='Humidity',
        tiles='CartoLight',
        frame_width=700,
        frame_height=500,
        line_color='blue',
        line_width=1,
        colorbar=True,
        hover_cols=['Humidity']
    )

# Create the map
city_map = create_city_map(map_data)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 20) & 
                                (city_data_df['Max Temp'] <= 25) &
                                (city_data_df['Humidity'] < 70)]
# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,wenling,28.3669,121.3600,23.74,58,65,3.49,CN,1698379181
45,45,avarua,-21.2078,-159.7750,22.03,64,75,4.12,CK,1698379183
69,69,al hazm,16.1641,44.7769,22.57,67,95,1.05,YE,1698379190
71,71,katsuura,35.1333,140.3000,21.77,54,4,5.93,JP,1698379190
84,84,deir ez-zor,35.3359,40.1408,21.34,50,95,4.21,SY,1698379194


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
37,wenling,CN,28.3669,121.3600,58,
45,avarua,CK,-21.2078,-159.7750,64,
69,al hazm,YE,16.1641,44.7769,67,
71,katsuura,JP,35.1333,140.3000,54,
84,deir ez-zor,SY,35.3359,40.1408,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    "radius": radius,
    "type": "hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wenling - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found
katsuura - nearest hotel: No hotel found
deir ez-zor - nearest hotel: No hotel found
hurghada - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
henderson - nearest hotel: No hotel found
upington - nearest hotel: No hotel found
yanchep - nearest hotel: No hotel found
brak - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
siwa oasis - nearest hotel: No hotel found
goondiwindi - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
muzaffarabad - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
rio claro - nearest hotel: No hotel found
hadithah - nearest hotel: No hotel found
lashkar gah - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
shingu - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
37,wenling,CN,28.3669,121.3600,58,No hotel found
45,avarua,CK,-21.2078,-159.7750,64,No hotel found
69,al hazm,YE,16.1641,44.7769,67,No hotel found
71,katsuura,JP,35.1333,140.3000,54,No hotel found
84,deir ez-zor,SY,35.3359,40.1408,50,No hotel found
110,hurghada,EG,27.2574,33.8129,46,No hotel found
116,tazacorte,ES,28.6290,-17.9293,68,No hotel found
132,henderson,US,36.0397,-114.9819,35,No hotel found
186,upington,ZA,-28.4478,21.2561,19,No hotel found
194,yanchep,AU,-31.5500,115.6833,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)